# **Week 4 - Capstone Development**

#### **Logistic Regression and Feature Scaling:**

So this is awkward, but I already did Logistic Regression and feature scaling in week 2. Furthermore, in week 3, I did a forward feature selection; a backward feature selection; and then a joined dataset where the numerical columns were seperated and then a PCA was performed on the data. From there, they were paired back up with the one-hot encoded data. 

In this week, we will find the featureset to rule them all :) 

In [1]:
# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns
import matplotlib.pyplot as plt



# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV,
    RepeatedStratifiedKFold,
    RepeatedKFold
)

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, root_mean_squared_error, accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_predict, KFold
# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

#### **Dataset Imports**

In [11]:
BDB_All_Plays_Model_Ready = pd.read_csv("/Users/leemcfarling/projects/ds_homework/AFL_Project/AFL_Final_Project/BDB_All_Plays_Model_Ready.csv") # Big Data Bowl Dataset

In [ ]:
# PDA_PCA_Features = pd.read_csv('../../Feature_Subsets/PDA_PCA_Features.csv')
# FNF_PCA_Features = pd.read_csv('../../Feature_Subsets/FNF_PCA_Features.csv')
# BDB_PCA_Features = pd.read_csv('../../Feature_Subsets/BDB_PCA_Features.csv')

# # === Backward Feature Sets ===
# FNF_back_Features = pd.read_csv('../../Feature_Subsets/FNF_back_Features.csv')
# PDA_back_Features = pd.read_csv('../../Feature_Subsets/PDA_back_Features.csv')
# BDB_back_Features = pd.read_csv('../../Feature_Subsets/BDB_back_Features.csv')

# === Forward Feature Sets ===
# FNF_Forward_Features = pd.read_csv('../../Feature_Subsets/FNF_Forward_Features.csv')
# PDA_Forward_Features = pd.read_csv('../../Feature_Subsets/PDA_Forward_Features.csv')
# BDB_Forward_Features = pd.read_csv('../../Feature_Subsets/BDB_Forward_Features.csv')

____

#### **Function Definitions**

Function to take a provided dataframe and split that dataframe into feature and target columns. 

In [5]:
# ===========================================================================================
# Function taken from Module 3 Final Project
# https://github.com/LeeMcFarling/Final_Project_Writeup/blob/main/Final_Project_Report.ipynb
# ===========================================================================================

def train_test_split_data(df, target_col):
    X = df.drop(columns=target_col)
    y = df[target_col]
# 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

#### **Data Pre-Processing and Splitting** 

So in week three, we already standardized the numerical features (and then did forward, or backward selection, etc.) and from there, we combined them back with the categorical features. 

In [6]:
PDA_PCA_X_train, PDA_PCA_X_test, PDA_PCA_y_train, PDA_PCA_y_test = train_test_split_data(PDA_PCA_Features, 'Inj_Occured')
FNF_PCA_X_train, FNF_PCA_X_test, FNF_PCA_y_train, FNF_PCA_y_test = train_test_split_data(FNF_PCA_Features, 'Inj_Occured')
BDB_PCA_X_train, BDB_PCA_X_test, BDB_PCA_y_train, BDB_PCA_y_test = train_test_split_data(BDB_PCA_Features, 'Inj_Occured')

FNF_back_X_train, FNF_back_X_test, FNF_back_y_train, FNF_back_y_test = train_test_split_data(FNF_back_Features, 'Inj_Occured')
PDA_back_X_train, PDA_back_X_test, PDA_back_y_train, PDA_back_y_test = train_test_split_data(PDA_back_Features, 'Inj_Occured') 
BDB_back_X_train, BDB_back_X_test, BDB_back_y_train, BDB_back_y_test = train_test_split_data(BDB_back_Features, 'Inj_Occured')


FNF_Forward_X_train, FNF_Forward_X_test, FNF_Forward_y_train, FNF_Forward_y_test = train_test_split_data(FNF_Forward_Features, 'Inj_Occured')
PDA_Forward_X_train, PDA_Forward_X_test, PDA_Forward_y_train, PDA_Forward_y_test = train_test_split_data(PDA_Forward_Features, 'Inj_Occured')
BDB_Forward_X_train, BDB_Forward_X_test, BDB_Forward_y_train, BDB_Forward_y_test = train_test_split_data(BDB_Forward_Features, 'Inj_Occured') 

____

#### **Run Model Classifier**

In [8]:
# =============================================================================================
# Taken from Mod 3 Week 8:
# https://github.com/waysnyder/Module-3-Assignments/blob/main/Homework_08.ipynb
# 
# Global dataframe logic taken from mod 3 final project: 
# https://github.com/LeeMcFarling/Final_Project_Writeup/blob/main/Final_Project_Report.ipynb
# 
# Final Function was developed in Week 2 of this Module
# =============================================================================================

def run_model_classifier(model, X_train, y_train, X_test, y_test, n_repeats=10, n_jobs=-1, run_comment=None, return_model=False, concat_results=False, **model_params):

    global combined_results
    # Remove extra key used to store error metric, if it was added to the parameter dictionary
    if 'accuracy_found' in model_params:
        model_params = model_params.copy()
        model_params.pop('accuracy_found', None)  
        
    # Instantiate the model if a class is provided
    if isinstance(model, type):
        model = model(**model_params)
    else:                                    
        model.set_params(**model_params)    

    model_name = model.__name__ if isinstance(model, type) else model.__class__.__name__ # Added because 


    # Use RepeatedStratifiedKFold for classification to preserve class distribution
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=n_repeats, random_state=42)
    
    # Perform 5-fold cross-validation using accuracy as the scoring metric
    cv_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=n_jobs)
    
    mean_cv_accuracy = np.mean(cv_scores)
    std_cv_accuracy  = np.std(cv_scores)
    
    # Fit the model on the full training set
    model.fit(X_train, y_train)
    
    # Compute training and testing accuracy
    train_preds    = model.predict(X_train)
    test_preds     = model.predict(X_test)

    # Normal Accuracy 
    train_accuracy = accuracy_score(y_train, train_preds)
    test_accuracy  = accuracy_score(y_test, test_preds)

    # Balanced Accuracy Metrics
    balanced_train_accuracy = balanced_accuracy_score(y_train, train_preds)
    balanced_test_accuracy = balanced_accuracy_score(y_test, test_preds)

    results_df = pd.DataFrame([{
        'model': model_name, 
        'model_params': model.get_params(),
        'mean_cv_accuracy': mean_cv_accuracy,
        'std_cv_accuracy': std_cv_accuracy,
        'train_accuracy': train_accuracy, 
        'test_accuracy': test_accuracy,
        'balanced_train_accuracy' : balanced_train_accuracy,
        'balanced_test_accuracy': balanced_test_accuracy,
        'run_comment': run_comment
    }])
    
    if concat_results:
        try:
            combined_results = pd.concat([combined_results, results_df], ignore_index=True)
        except NameError:
            combined_results = results_df

    return (results_df, model) if return_model else results_df

_____

# **Modeling**

NOTE: As previously discussed in Semester 2, the primary goal of this analysis and modeling excercise is to classify whether a particular play will result in an injury, and to determine the factors that are most likely to cause this injury. Furthermore, as was *also* previously discussed, this dataset has some extreme imbalance issues (injury occurance < 2%), and as such high performance on these baseline models is NOT expected. 

## **Baseline Logistic Regression**

#to do: **put baseline here**

So, in order to help convergence, we cranked the max_iter, and tol hyperparameters. 

____

In [ ]:
PDA_PCA_X_train, PDA_PCA_X_test, PDA_PCA_y_train, PDA_PCA_y_test = train_test_split_data(PDA_PCA_Features, 'Inj_Occured')
FNF_PCA_X_train, FNF_PCA_X_test, FNF_PCA_y_train, FNF_PCA_y_test = train_test_split_data(FNF_PCA_Features, 'Inj_Occured')
BDB_PCA_X_train, BDB_PCA_X_test, BDB_PCA_y_train, BDB_PCA_y_test = train_test_split_data(BDB_PCA_Features, 'Inj_Occured')

FNF_back_X_train, FNF_back_X_test, FNF_back_y_train, FNF_back_y_test = train_test_split_data(FNF_back_Features, 'Inj_Occured')
PDA_back_X_train, PDA_back_X_test, PDA_back_y_train, PDA_back_y_test = train_test_split_data(PDA_back_Features, 'Inj_Occured') 
BDB_back_X_train, BDB_back_X_test, BDB_back_y_train, BDB_back_y_test = train_test_split_data(BDB_back_Features, 'Inj_Occured')


# FNF_Forward_X_train, FNF_Forward_X_test, FNF_Forward_y_train, FNF_Forward_y_test = train_test_split_data(FNF_Forward_Features, 'Inj_Occured')
# PDA_Forward_X_train, PDA_Forward_X_test, PDA_Forward_y_train, PDA_Forward_y_test = train_test_split_data(PDA_Forward_Features, 'Inj_Occured')
# BDB_Forward_X_train, BDB_Forward_X_test, BDB_Forward_y_train, BDB_Forward_y_test = train_test_split_data(BDB_Forward_Features, 'Inj_Occured') 

## **Forward Feature Selection Logistic Regression**

First we will check performance on forward feature selection using a 5 fold cross validation

#### **Big Data Bowl**

In [9]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

BDB_F_results_df = run_model_classifier(
    LogisticRegression,
    BDB_Forward_X_train, 
    BDB_Forward_y_train, 
    BDB_Forward_X_test,
    BDB_Forward_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='BDB Forward Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

BDB_F_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.666675,0.027941,0.694254,0.704678,0.652935,0.59259,BDB Forward Features - Logistic_Regression


#### **Punt Data Analytics**

In [11]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

PDA_F_results_df = run_model_classifier(
    LogisticRegression,
    PDA_Forward_X_train, 
    PDA_Forward_y_train, 
    PDA_Forward_X_test,
    PDA_Forward_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='PDA Forward Features - Logistic_Regression', 
    return_model=False,
    concat_results=False,
    **params_
    )

PDA_F_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.600339,0.066292,0.69985,0.706058,0.670351,0.618873,PDA Forward Features - Logistic_Regression


#### **First and Future**

In [12]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

FNF_F_results_df = run_model_classifier(
    LogisticRegression,
    FNF_Forward_X_train, 
    FNF_Forward_y_train, 
    FNF_Forward_X_test,
    FNF_Forward_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='FNF Forward Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

FNF_F_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.553176,0.052417,0.50154,0.504925,0.661168,0.602444,FNF Forward Features - Logistic_Regression


____

## **Forward Feature Selection Results**

In [19]:
combined_results[combined_results['run_comment'].str.contains('Forward')].sort_values(by='mean_cv_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.666675,0.027941,0.694254,0.704678,0.652935,0.592590,BDB Forward Features - Logistic_Regression
1,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.600339,0.066292,0.699850,0.706058,0.670351,0.618873,PDA Forward Features - Logistic_Regression
2,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.553176,0.052417,0.501540,0.504925,0.661168,0.602444,FNF Forward Features - Logistic_Regression


____

## **Backward Feature Selection + Logistic Regression Tasks**

#### **Big Data Bowl**

In [ ]:
FNF_back_X_train, FNF_back_X_test, FNF_back_y_train, FNF_back_y_test = train_test_split_data(FNF_back_Features, 'Inj_Occured')
PDA_back_X_train, PDA_back_X_test, PDA_back_y_train, PDA_back_y_test = train_test_split_data(PDA_back_Features, 'Inj_Occured') 


In [13]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

BDB_B_results_df = run_model_classifier(
    LogisticRegression,
    BDB_back_X_train, 
    BDB_back_y_train, 
    BDB_back_X_test,
    BDB_back_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='BDB Backward Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

BDB_B_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.620591,0.14278,0.691475,0.676608,0.680347,0.566018,BDB Backward Features - Logistic_Regression


#### **Punt Data Analytics Backward Selection**

In [14]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

PDA_B_results_df = run_model_classifier(
    LogisticRegression,
    PDA_back_X_train, 
    PDA_back_y_train, 
    PDA_back_X_test,
    PDA_back_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='PDA Backward Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

PDA_B_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.620609,0.052367,0.651946,0.658938,0.665773,0.624042,PDA Backward Features - Logistic_Regression


#### **First and Future Backward Selection Logistic Regression**

In [15]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

FNF_B_results_df = run_model_classifier(
    LogisticRegression,
    FNF_back_X_train, 
    FNF_back_y_train, 
    FNF_back_X_test,
    FNF_back_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='FNF Backward Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

FNF_B_results_df

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.562554,0.091125,0.56859,0.570728,0.63502,0.635352,FNF Backward Features - Logistic_Regression


____

#### **Backward Selection Results**

In [20]:
combined_results[combined_results['run_comment'].str.contains('Backward')].sort_values(by='mean_cv_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
4,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.620609,0.052367,0.651946,0.658938,0.665773,0.624042,PDA Backward Features - Logistic_Regression
3,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.620591,0.142780,0.691475,0.676608,0.680347,0.566018,BDB Backward Features - Logistic_Regression
5,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.562554,0.091125,0.568590,0.570728,0.635020,0.635352,FNF Backward Features - Logistic_Regression


___

## **PC(Logistic)Regression**

#### **Big Data Bowl**

In [21]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

BDB_PCA_results_df = run_model_classifier(
    LogisticRegression,
    BDB_PCA_X_train, 
    BDB_PCA_y_train, 
    BDB_PCA_X_test,
    BDB_PCA_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='BDB PCA Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

BDB_PCA_results_df

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.697148,0.016413,0.697324,0.681871,0.729483,0.50771,BDB PCA Features - Logistic_Regression


#### **First and Future**

In [26]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 5000,                  # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

FNF_PCA_results_df = run_model_classifier(
    LogisticRegression,
    FNF_PCA_X_train, 
    FNF_PCA_y_train, 
    FNF_PCA_X_test,
    FNF_PCA_y_test,
    n_repeats=3, 
    n_jobs=-1, 
    run_comment='FNF PCA Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

FNF_PCA_results_df

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lee

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.610246,0.074868,0.574371,0.576664,0.58569,0.738302,FNF PCA Features - Logistic_Regression


#### **Punt Data Analytics**

In [24]:
params_ = {
    'class_weight' : 'balanced',        # attempt to balance dataset
    'solver': 'saga',                   # Doc said that this solver is better for larger datasets
    'penalty': 'l2',                    # default
    'max_iter' : 50000,                 # Iteratively increased this until Convergence Warnings went away
    'tol': 1e-2,                        # Another convergence warning measure
    'random_state' : 42
}

PDA_PCA_results_df = run_model_classifier(
    LogisticRegression,
    PDA_PCA_X_train, 
    PDA_PCA_y_train, 
    PDA_PCA_X_test,
    PDA_PCA_y_test,
    n_repeats=5, 
    n_jobs=-1, 
    run_comment='PDA PCA Features - Logistic_Regression', 
    return_model=False,
    concat_results=True,
    **params_
    )

PDA_PCA_results_df

/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/leemcfarling/venvs/tfclean/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lee

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.643218,0.138579,0.439746,0.458489,0.623881,0.55156,PDA PCA Features - Logistic_Regression


____

## **PCA'd Numeric Data - Results**

In [27]:
combined_results[combined_results['run_comment'].str.contains('PCA')].sort_values(by='mean_cv_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
6,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.697148,0.016413,0.697324,0.681871,0.729483,0.507710,BDB PCA Features - Logistic_Regression
7,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.643218,0.138579,0.439746,0.458489,0.623881,0.551560,PDA PCA Features - Logistic_Regression
8,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.610246,0.074868,0.574371,0.576664,0.585690,0.738302,FNF PCA Features - Logistic_Regression


____

## **Best Results for Each Dataset + Discussion**

So just before we get started, I wanted to point out that the Logistic regression model was really struggling with standardized featueres (PCA). In this I mean it was running for a long period of time, spat out a bunch of convergence warnings, etc. 

This is very aligned with how this has been behaving so far for logistic regression, and as stated previously, I think that it's a signal that different models will actually be better moving forward. In terms of results, in each individual run, the Big Data Bowl dataset seemed more predictive in terms of cross validation than the other datasets (which is incredibly ironic considering the other two datasets were specifically looking at injuries whereas the big databowl was more of a generalized thing). But there was a bit of a discrepency between Model+Featuresets that performed best on cross validation, and which ones performed best on 'balanced accuracy' in the test set. 

In terms of the cross validation, despite the convergence warnings, the model did the best on the principle components of the numeric data vs the forward and backward selected featuresets. (This might just be that the numeric data in this case was more standardized in each of these scenarios). But each dataset had a different 'featureset' that performed best when it came to the balanced accuracy of the test set. This is indicative of the model possibly either overfitting slightly on the cross validation folds, or that the cross validation folds weren't reliably representative of the test set (which in our case, since there is an extreme class imbalance in the target variable, usually 98% to 2% or higher), that could definitely be a possibility. 

Becuause I'm still testing other models, I think I'll just take the PCA featureset forward for each of these datsets to test on the SVCs and the Decision Trees -- and I'm going to do this for a few reasons -- Because it's more indicative of the overall data, because i'm expecting those models to perform much better on this task than the baseline logistic regression, and because, (if they behave how I would expect) I would re-do the feature selection process with the new model as the feature selector to eek out the best performance anyways. 


#### **Big Data Bowl**

In [31]:
combined_results[combined_results['run_comment'].str.contains('BDB')].sort_values(by='balanced_test_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.666675,0.027941,0.694254,0.704678,0.652935,0.592590,BDB Forward Features - Logistic_Regression
3,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.620591,0.142780,0.691475,0.676608,0.680347,0.566018,BDB Backward Features - Logistic_Regression
6,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.697148,0.016413,0.697324,0.681871,0.729483,0.507710,BDB PCA Features - Logistic_Regression


#### **First and Future**

In [32]:
combined_results[combined_results['run_comment'].str.contains('FNF')].sort_values(by='balanced_test_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
8,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.610246,0.074868,0.574371,0.576664,0.585690,0.738302,FNF PCA Features - Logistic_Regression
5,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.562554,0.091125,0.568590,0.570728,0.635020,0.635352,FNF Backward Features - Logistic_Regression
2,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.553176,0.052417,0.501540,0.504925,0.661168,0.602444,FNF Forward Features - Logistic_Regression


#### **Punt Data Analytics**

In [33]:
combined_results[combined_results['run_comment'].str.contains('PDA')].sort_values(by='balanced_test_accuracy', ascending=False)

,model,model_params,mean_cv_accuracy,std_cv_accuracy,train_accuracy,test_accuracy,balanced_train_accuracy,balanced_test_accuracy,run_comment
4,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.620609,0.052367,0.651946,0.658938,0.665773,0.624042,PDA Backward Features - Logistic_Regression
1,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.600339,0.066292,0.699850,0.706058,0.670351,0.618873,PDA Forward Features - Logistic_Regression
7,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.643218,0.138579,0.439746,0.458489,0.623881,0.551560,PDA PCA Features - Logistic_Regression


_____